In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar3/"
use_tfidfvectorizer = True
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 50
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.4.6"
merge_list = ["t2.4.3", "t2.4.5"]


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.6.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

9042

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t2.4.6+2.4.3+2.4.5


In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.3.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.5.pkl


In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

9042

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 9042
topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

0.03687596321105957

In [18]:
word_doc_matrix

<9042x556 sparse matrix of type '<class 'numpy.float64'>'
	with 49599 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 4674 edges.
It has only fully disconnected 36 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/500 [00:00]

CPU times: user 1min 56s, sys: 1.31 s, total: 1min 57s
Wall time: 1min 12s


In [21]:
print(runtime_umap)

72.1311457157135


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/100 [00:00]

CPU times: user 25.9 s, sys: 79.7 ms, total: 26 s
Wall time: 22.8 s


In [23]:
print(runtime_umap_transform)

22.82575011253357


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_embedding_t2.4.6+2.4.3+2.4.5.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_clusterable_embedding_t2.4.6+2.4.3+2.4.5.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.1635,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
3,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs50,50,200,0.265218,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
3,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs50,50,200,0.265218,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
4,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs50,50,250,0.302413,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
3,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs50,50,200,0.265218,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
4,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs50,50,250,0.302413,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs450,450,50,0.179052,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
73,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs450,450,100,0.201868,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
74,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs450,450,150,0.237690,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
75,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs450,450,200,0.275418,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
3,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs50,50,200,0.265218,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
4,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs50,50,250,0.302413,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs450,450,100,0.201868,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
74,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs450,450,150,0.237690,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
75,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs450,450,200,0.275418,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
76,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs450,450,250,0.305602,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
3,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs50,50,200,0.265218,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
4,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs50,50,250,0.302413,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs450,450,150,0.237690,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
75,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs450,450,200,0.275418,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
76,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs450,450,250,0.305602,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
77,hdb,t2.4.6+2.4.3+2.4.5,1,ss300cs450,450,300,0.337981,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
3,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs50,50,200,0.265218,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
4,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs50,50,250,0.302413,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs450,450,200,0.275418,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
76,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs450,450,250,0.305602,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
77,hdb,t2.4.6+2.4.3+2.4.5,1,ss300cs450,450,300,0.337981,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
78,hdb,t2.4.6+2.4.3+2.4.5,1,ss350cs450,450,350,0.374617,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.6+2.4.3+2.4.5,1,ss50cs50,50,50,0.163500,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
1,hdb,t2.4.6+2.4.3+2.4.5,1,ss100cs50,50,100,0.195185,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
2,hdb,t2.4.6+2.4.3+2.4.5,1,ss150cs50,50,150,0.232725,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
3,hdb,t2.4.6+2.4.3+2.4.5,1,ss200cs50,50,200,0.265218,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
4,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs50,50,250,0.302413,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.4.6+2.4.3+2.4.5,1,ss250cs450,450,250,0.305602,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
77,hdb,t2.4.6+2.4.3+2.4.5,1,ss300cs450,450,300,0.337981,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
78,hdb,t2.4.6+2.4.3+2.4.5,1,ss350cs450,450,350,0.374617,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."
79,hdb,t2.4.6+2.4.3+2.4.5,1,ss400cs450,450,400,0.415985,"rt_vect: 0.03687596321105957, min_df: 4, max_d...","hellinger, rt_umap_tf: 22.82575011253357, rt_u..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,google/chaps-linux,26,3,1,1,-1,-1,-1,-1,-1,...,-1,3,4,-1,1,-1,-1,-1,-1,-1
1,Azure/ai-solution-accelerators-list,15,3,1,1,-1,-1,-1,-1,-1,...,-1,0,0,1,1,-1,-1,-1,-1,-1
2,cbucher/console,23,3,1,1,1,-1,-1,-1,-1,...,-1,1,1,0,1,0,-1,-1,-1,-1
3,intel/ipp-crypto,32,3,1,1,-1,-1,-1,-1,-1,...,-1,3,4,3,1,-1,-1,-1,-1,-1
4,WILL-Hunting/FakeXiecheng.API,-1,3,1,1,5,5,-1,-1,-1,...,-1,3,-1,3,1,2,2,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9037,exoplanet-imaging-challenge/exoplanet-imaging-...,24,3,1,1,4,3,4,4,3,...,2,2,3,3,1,2,2,2,2,2
9038,samuel-code/face-detection,9,-1,1,1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9039,ApiO/jquery.triSwitch,2,0,0,0,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9040,mohammaduzair9/Face-Recognition,30,3,1,1,-1,1,-1,-1,-1,...,-1,3,-1,-1,0,-1,0,-1,-1,-1


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_labels_results_t2.4.6+2.4.3+2.4.5.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
3,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,19,36,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.145521,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
3,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,19,36,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.145521,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
4,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,22,36,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.169650,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


...

,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
3,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,19,36,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.145521,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
4,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,22,36,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.169650,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
5,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,25,36,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.175381,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
6,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,28,36,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.197743,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
7,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,31,36,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.213581,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
8,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,34,36,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.223933,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
9,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,37,36,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.243816,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
3,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,19,36,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.145521,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
4,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,22,36,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.169650,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
5,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,25,36,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.175381,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
6,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,28,36,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.197743,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
7,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,31,36,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.213581,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
8,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,34,36,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.223933,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
9,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,37,36,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.243816,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
3,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,19,36,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.145521,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
4,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,22,36,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.169650,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
5,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,25,36,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.175381,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
6,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,28,36,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.197743,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
7,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,31,36,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.213581,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
8,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,34,36,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.223933,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
9,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,37,36,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.243816,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
3,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,19,36,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.145521,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
4,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,22,36,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.169650,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
5,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,25,36,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.175381,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
6,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,28,36,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.197743,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
7,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,31,36,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.213581,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
8,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,34,36,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.223933,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
9,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,37,36,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.243816,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,10,36,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.174549,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
1,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,13,36,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.104973,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
2,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,16,36,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.114786,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
3,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,19,36,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.145521,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
4,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,22,36,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.169650,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
5,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,25,36,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.175381,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
6,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,28,36,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.197743,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
7,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,31,36,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.213581,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
8,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,34,36,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.223933,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."
9,kmeans-umapt,t2.4.6+2.4.3+2.4.5,1,37,36,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",0.243816,"(hellinger, rt_umap_tf: 22.82575011253357, rt_..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,google/chaps-linux,4,0,1,10,18,6,26,5,21,23,19,35,28,39
1,Azure/ai-solution-accelerators-list,1,7,10,0,6,5,14,12,0,1,31,41,16,2
2,cbucher/console,0,3,0,11,2,9,25,22,13,8,27,27,4,0
3,intel/ipp-crypto,4,0,1,10,18,23,26,30,21,23,19,24,7,39
4,WILL-Hunting/FakeXiecheng.API,4,0,1,10,16,6,0,23,16,33,1,11,38,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9037,exoplanet-imaging-challenge/exoplanet-imaging-...,9,3,0,3,13,0,9,0,7,21,38,23,14,16
9038,samuel-code/face-detection,4,0,15,12,1,20,21,19,26,6,12,39,18,28
9039,ApiO/jquery.triSwitch,2,8,3,5,3,2,2,3,12,4,2,3,2,4
9040,mohammaduzair9/Face-Recognition,8,11,5,9,5,1,10,29,15,35,26,17,1,3


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_labels_results_t2.4.6+2.4.3+2.4.5.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244
3,kmeans,t2.4.6+2.4.3+2.4.5,1,19,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.329382


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244
3,kmeans,t2.4.6+2.4.3+2.4.5,1,19,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.329382
4,kmeans,t2.4.6+2.4.3+2.4.5,1,22,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.910802


...

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244
3,kmeans,t2.4.6+2.4.3+2.4.5,1,19,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.329382
4,kmeans,t2.4.6+2.4.3+2.4.5,1,22,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.910802
5,kmeans,t2.4.6+2.4.3+2.4.5,1,25,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.722993
6,kmeans,t2.4.6+2.4.3+2.4.5,1,28,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.941095
7,kmeans,t2.4.6+2.4.3+2.4.5,1,31,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.846045
8,kmeans,t2.4.6+2.4.3+2.4.5,1,34,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.859257
9,kmeans,t2.4.6+2.4.3+2.4.5,1,37,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.999510


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244
3,kmeans,t2.4.6+2.4.3+2.4.5,1,19,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.329382
4,kmeans,t2.4.6+2.4.3+2.4.5,1,22,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.910802
5,kmeans,t2.4.6+2.4.3+2.4.5,1,25,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.722993
6,kmeans,t2.4.6+2.4.3+2.4.5,1,28,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.941095
7,kmeans,t2.4.6+2.4.3+2.4.5,1,31,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.846045
8,kmeans,t2.4.6+2.4.3+2.4.5,1,34,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.859257
9,kmeans,t2.4.6+2.4.3+2.4.5,1,37,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.999510


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244
3,kmeans,t2.4.6+2.4.3+2.4.5,1,19,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.329382
4,kmeans,t2.4.6+2.4.3+2.4.5,1,22,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.910802
5,kmeans,t2.4.6+2.4.3+2.4.5,1,25,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.722993
6,kmeans,t2.4.6+2.4.3+2.4.5,1,28,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.941095
7,kmeans,t2.4.6+2.4.3+2.4.5,1,31,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.846045
8,kmeans,t2.4.6+2.4.3+2.4.5,1,34,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.859257
9,kmeans,t2.4.6+2.4.3+2.4.5,1,37,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.999510


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244
3,kmeans,t2.4.6+2.4.3+2.4.5,1,19,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.329382
4,kmeans,t2.4.6+2.4.3+2.4.5,1,22,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.910802
5,kmeans,t2.4.6+2.4.3+2.4.5,1,25,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.722993
6,kmeans,t2.4.6+2.4.3+2.4.5,1,28,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.941095
7,kmeans,t2.4.6+2.4.3+2.4.5,1,31,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.846045
8,kmeans,t2.4.6+2.4.3+2.4.5,1,34,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.859257
9,kmeans,t2.4.6+2.4.3+2.4.5,1,37,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.999510


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.6+2.4.3+2.4.5,1,10,10,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.516861
1,kmeans,t2.4.6+2.4.3+2.4.5,1,13,13,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.388624
2,kmeans,t2.4.6+2.4.3+2.4.5,1,16,16,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.446244
3,kmeans,t2.4.6+2.4.3+2.4.5,1,19,19,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.329382
4,kmeans,t2.4.6+2.4.3+2.4.5,1,22,22,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.910802
5,kmeans,t2.4.6+2.4.3+2.4.5,1,25,25,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.722993
6,kmeans,t2.4.6+2.4.3+2.4.5,1,28,28,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.941095
7,kmeans,t2.4.6+2.4.3+2.4.5,1,31,31,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.846045
8,kmeans,t2.4.6+2.4.3+2.4.5,1,34,34,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.859257
9,kmeans,t2.4.6+2.4.3+2.4.5,1,37,37,"rt_vect: 0.03687596321105957, min_df: 4, max_d...",1.999510


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,google/chaps-linux,7,6,1,3,14,9,20,15,13,21,18,0,33,1
1,Azure/ai-solution-accelerators-list,0,8,7,15,17,3,12,1,11,3,26,42,0,43
2,cbucher/console,4,5,3,12,12,10,27,2,7,13,34,33,1,37
3,intel/ipp-crypto,9,12,14,14,6,13,26,18,0,30,31,15,12,27
4,WILL-Hunting/FakeXiecheng.API,1,6,1,0,1,8,15,22,4,14,6,7,35,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9037,exoplanet-imaging-challenge/exoplanet-imaging-...,4,5,3,12,12,10,8,9,7,2,34,3,11,31
9038,samuel-code/face-detection,5,0,12,17,10,11,22,11,27,12,13,41,27,44
9039,ApiO/jquery.triSwitch,8,9,5,18,19,20,7,21,31,35,15,2,17,41
9040,mohammaduzair9/Face-Recognition,4,6,1,3,2,23,16,19,16,13,16,6,28,23


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_labels_results_t2.4.6+2.4.3+2.4.5.pkl
